In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
trainFile = 'round1_ijcai_18_train_20180301.txt'
testFile = 'round1_ijcai_18_test_a_20180301.txt'
dataDir = '../../Data/advertisment/Raw/'

In [3]:
dfTrain = pd.read_table(dataDir+trainFile,sep=' ')
dfTest = pd.read_table(dataDir+testFile,sep=' ')

In [4]:
dfTrain.dtypes

instance_id                    int64
item_id                        int64
item_category_list            object
item_property_list            object
item_brand_id                  int64
item_city_id                   int64
item_price_level               int64
item_sales_level               int64
item_collected_level           int64
item_pv_level                  int64
user_id                        int64
user_gender_id                 int64
user_age_level                 int64
user_occupation_id             int64
user_star_level                int64
context_id                     int64
context_timestamp              int64
context_page_id                int64
predict_category_property     object
shop_id                        int64
shop_review_num_level          int64
shop_review_positive_rate    float64
shop_star_level                int64
shop_score_service           float64
shop_score_delivery          float64
shop_score_description       float64
is_trade                       int64
d

In [5]:
dfTrain.head(5)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [40]:
dfTrain['predict_category_property_bin'] = dfTrain['predict_category_property'].apply(lambda x: x.count(';'))
dfTrain['predict_category_property_bin'].describe()

count    478138.000000
mean          3.811791
std           1.797422
min           0.000000
25%           2.000000
50%           4.000000
75%           5.000000
max          13.000000
Name: predict_category_property_bin, dtype: float64

In [45]:
dfTrain.loc[dfTrain['predict_category_property_bin']==0,'predict_category_property']

163                 7908382889764677758:8500395963709890328
333                 7908382889764677758:2636395404473730413
970                 7908382889764677758:2636395404473730413
1303                7908382889764677758:8500395963709890328
1606                7908382889764677758:8500395963709890328
1950      836752724084922533:5131280576272319091,8218883...
2103                                 7908382889764677758:-1
2568                7908382889764677758:2636395404473730413
3641                                 7908382889764677758:-1
3769      509660095530134768:1885623735035256840,5514263...
3784                                 7908382889764677758:-1
3811                7908382889764677758:2636395404473730413
4618                                 7908382889764677758:-1
6176      836752724084922533:2636395404473730413,8218883...
6779                7908382889764677758:1326487468817980284
7773                7908382889764677758:2636395404473730413
8717                                 790

In [16]:
for i in range(3):
    print(dfTrain['item_category_list_bin_%d'%i].unique())

['7908382889764677758']
['5799347067982556520' '8277336076276184272' '5755694407684602296'
 '4879721024980945592' '509660095530134768' '2436715285093487584'
 '7258015885215914736' '8710739180200009128' '2011981573061447208'
 '22731265849056483' '3203673979138763595' '2642175453151805566'
 '1968056100269760729']
[' ' '8868887661186419229' '6233669177166538628']


In [22]:
tt = np.array(';'.join(dfTrain['item_property_list'].tolist()).split(';'))

In [26]:
np.unique(tt).shape

(61407,)

In [39]:
dfTrain['shop_id'].value_counts()[dfTrain['shop_id'].value_counts()>1].index

Int64Index([6597981382309269962, 1932086538588242822, 8953967780295706657,
            2025576994711098052, 4215742678121737516, 4300400395541925383,
            5813143468854095050, 4954894036019286224, 5965128262672848132,
            4413707559864701260,
            ...
            5394523850932456084, 7233126228408773348, 1329555954568296183,
            8492909981978620547, 3190575743059897408,  491170600600448045,
            6147737886549892275, 8583017644155961089, 6419968885868975562,
            9208074232025895234],
           dtype='int64', length=3502)

In [29]:
dfTrain.loc[dfTrain['user_id']==7201324926905945581,'is_trade']

0

In [34]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

dfTrain['context_timestamp_date'] = dfTrain['context_timestamp'].map(timestamp_datetime).apply(lambda x: x[:10])
dfTest['context_timestamp_date'] = dfTest['context_timestamp'].map(timestamp_datetime).apply(lambda x: x[:10])

In [35]:
dfTrain['context_timestamp_date'].value_counts()

2018-09-18    78268
2018-09-21    71199
2018-09-19    70931
2018-09-20    68387
2018-09-22    68318
2018-09-23    63614
2018-09-24    57421
Name: context_timestamp_date, dtype: int64

In [33]:
print(dfTrain['context_timestamp_date'].max())
print(dfTrain['context_timestamp_date'].min())
print(dfTest['context_timestamp_date'].max())
print(dfTest['context_timestamp_date'].min())

2018-09-24 23:59:47
2018-09-18 00:00:01
2018-09-25 23:59:25
2018-09-25 00:00:02


In [36]:
dfTest.shape

(18371, 27)

In [15]:
for i in dfTrain['predict_category_property'].tolist()[:10]:
    tmpDict ={}
    for j in i.split(';'):
        m,n = j.split(':')
        tmpDict[m] = n.split(',')

In [39]:
def dictgen(i):
    tmpDict ={}
    for j in i.split(';'):
        try:
            m,n = j.split(':')
            '''if not '-1' in n.split(','):
                tmpDict[m] = len(n.split(','))'''
            tmpDict[m] = len(n.split(',')) if not '-1' in n.split(',') else -1
        except: 
                tmpDict ={'-1':'-1'}
    return tmpDict

In [40]:
dfTrain['predict_category_property_dict'] = dfTrain['predict_category_property'].map(dictgen)

In [41]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=True)
X = v.fit_transform(dfTrain['predict_category_property_dict'].tolist())
X.shape

(478138, 1032)

In [38]:
dfTrain['predict_category_property_dict'].tolist()[0:10]

[{'8277336076276184272': 1},
 {'5799347067982556520': 1, '7908382889764677758': 5},
 {'5799347067982556520': 1,
  '7258015885215914736': 1,
  '7908382889764677758': 1},
 {'1950314698730389427': 7,
  '7492960463130085436': 1,
  '7908382889764677758': 2},
 {'5799347067982556520': 1, '7908382889764677758': 1},
 {'4879721024980945592': 1,
  '5799347067982556520': 1,
  '7908382889764677758': 2},
 {'2011981573061447208': 1,
  '7908382889764677758': 1,
  '8277336076276184272': 1},
 {'2011981573061447208': 1,
  '7908382889764677758': 1,
  '8277336076276184272': 1},
 {'7908382889764677758': 1},
 {'5799347067982556520': 2, '8277336076276184272': 1}]

In [44]:
idTrain = set(dfTrain.user_id.unique().tolist())
idTest = set(dfTest.user_id.unique().tolist())

In [47]:
print(len(idTrain))
print(len(idTest))
print(len(idTrain&idTest))


197694
13573
3626


In [48]:
dfTrain.shape

(478138, 27)

In [49]:
dfTest.shape

(18371, 26)